In [ ]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
ticker = yf.Ticker("RELI")
data=ticker.history(start="2022-01-01",end="2024-12-31",interval="1d")
data = data.dropna()
print(data.head())

                                  Open         High          Low        Close  \
Date                                                                            
2022-01-03 00:00:00-05:00  1660.050049  2435.250000  1606.500000  1932.900024   
2022-01-04 00:00:00-05:00  1782.449951  2572.949951  1736.550049  2376.600098   
2022-01-05 00:00:00-05:00  2152.199951  2674.949951  2073.149902  2269.500000   
2022-01-06 00:00:00-05:00  2286.840088  2437.800049  1657.500000  1721.250000   
2022-01-07 00:00:00-05:00  1731.449951  1884.449951  1448.400024  1555.500000   

                           Volume  Dividends  Stock Splits  
Date                                                        
2022-01-03 00:00:00-05:00  100015        0.0           0.0  
2022-01-04 00:00:00-05:00  204411        0.0           0.0  
2022-01-05 00:00:00-05:00   94822        0.0           0.0  
2022-01-06 00:00:00-05:00   25985        0.0           0.0  
2022-01-07 00:00:00-05:00   15810        0.0           0.0  


In [ ]:
#bb
def bb(data,window=20):
  data["sma"]= data["Close"].rolling(window).mean()
  data["std"]= data["Close"].rolling(window).std()
  data["upperband"]=data["sma"]+2*(data["std"])
  data["lowerband"]=data["sma"]-2*(data["std"])

  count_buy_bollinger=0
  count_sell_bollinger=0

  data.dropna(inplace=True)
  data['Buy_bollinger'] = np.where(data['Close'] > data['upperband'], 1, 0)
  data['Sell_bollinger'] = np.where(data['Close'] < data['lowerband'], -1, 0)

  return data
#macd
def macd(data,window_signal=9):
  data["ema_slow"]=data["Close"].ewm(span=26,adjust=False).mean()
  data["ema_fast"]=data["Close"].ewm(span=12,adjust=False).mean()
  data["signal"]=data["Close"].ewm(span=window_signal,adjust=False).mean()
  data["macd"]=data["ema_fast"]-data["ema_slow"]
  count_buy_macd=0
  count_sell_macd=0

  data['Buy_MACD']=np.where((data['macd']>data['signal'])& (data['macd'].shift(1) < data['signal'].shift(1)),1,0)
  data['Sell_MACD']=np.where((data['macd']<data['signal'])& (data['macd'].shift(1) > data['signal'].shift(1)),-1,0)

  for i in range(1, len(data)):
        if data['Buy_MACD'].iloc[i] == 1:
          count_buy_macd+=1
        else:
         count_sell_macd+=1
  print("count_buy_macd: ",count_buy_macd)
  print("count_sell_macd: ",count_sell_macd)


  return data

def rsi(data,period=14):
  delta = data['Close'].diff()
  gain = delta.where(delta > 0, 0)
  loss = -delta.where(delta < 0, 0)
  avg_gain = gain.rolling(window=period).mean()
  avg_loss = loss.rolling(window=period).mean()
  rs = avg_gain / avg_loss
  data['RSI'] = 100 - (100 / (1 + rs))
  count_buy_rsi=0
  count_sell_rsi=0


  data['Buy_RSI']=np.where(data['RSI']<30,1,0)
  data['Sell_RSI']=np.where(data['RSI']>70,-1,0)

  for i in range(1, len(data)):
        if data['Buy_RSI'].iloc[i] == 1:
          count_buy_rsi+=1
        else:
         count_sell_rsi+=1

  print("count_buy_rsi: ",count_buy_rsi)
  print("count_sell_rsi: ",count_sell_rsi)
  return data
#atr
def atr(data,period=14):
  data["h-l"] = data["High"]-data["Low"]
  data["h-pc"] = abs(data["High"]-data["Close"].shift(1))
  data["l-pc"] = abs(data["Low"]-data["Close"].shift(1))
  data["tr"]=data[["h-l","h-pc","l-pc"]].max(axis=1)
  data["atr"]=data["tr"].rolling(period).mean()
  # Count buy and sell ATR signals only
  data['ATR_signal'] = 0
  threshold = data['atr'].mean()
  data.loc[data['atr'] > threshold, 'ATR_signal'] = -1  # Sell
  data.loc[data['atr'] < threshold, 'ATR_signal'] = 1
  atr_counts = data['ATR_signal'].value_counts()
  count_buy_atr = atr_counts.get(1, 0)
  count_sell_atr = atr_counts.get(-1, 0)
  print("Count Buy ATR:", count_buy_atr)
  print("Count Sell ATR:", count_sell_atr)

  return data
def sc(data):
  period = 14
  low_min = data['Low'].rolling(window=period).min()
  high_max = data['High'].rolling(window=period).max()
  data['%K'] = 100 * (data['Close'] - low_min) / (high_max - low_min)
  data['%D'] = data['%K'].rolling(window=3).mean()
  count_buy_so=0
  count_sell_so=0
  data['Buy_D']=np.where(data['%D']<20,1,0)
  data['Sell_D']=np.where(data['%D']>80,-1,0)

  for i in range(1, len(data)):
        if data['Buy_D'].iloc[i] == 1:
          count_buy_so+=1
        else:
         count_sell_so+=1
  print("count_buy_so: ",count_buy_so)
  print("count_sell_so: ",count_sell_so)
  return data
def vwap(data):
  data['Typical_Price'] = (data['High'] + data['Low'] + data['Close']) / 3
  data['TPV'] = data['Typical_Price'] * data['Volume']
  data['Cum_TPV'] = data['TPV'].cumsum()
  data['Cum_Volume'] = data['Volume'].cumsum()
  data['VWAP'] = data['Cum_TPV'] / data['Cum_Volume']
  count_buy_vwap=0
  count_sell_vwap=0

  data['Buy_VWAP']=np.where((data['Close']>data['VWAP'])&((data['Close'].shift(1) < data['VWAP'].shift(1))),1,0)
  data['Sell_VWAP']=np.where((data['Close']<data['VWAP'])&(data['Close'].shift(1)>data['VWAP'].shift(1)),-1,0)

  for i in range(1, len(data)):
        if data['Buy_VWAP'].iloc[i] == 1:
          count_buy_vwap+=1
        else:
         count_sell_vwap+=1
  print("count_buy_vwap: ",count_buy_vwap)
  print("count_sell_vwap: ",count_sell_vwap)
  return data
  return data

vwap(data)
bb(data)
atr(data)
rsi(data)
macd(data)
sc(data)



count_buy_vwap:  3
count_sell_vwap:  729
Count Buy ATR: 546
Count Sell ATR: 155
count_buy_rsi:  221
count_sell_rsi:  492
count_buy_macd:  0
count_sell_macd:  713
count_buy_so:  351
count_sell_so:  362


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Typical_Price,TPV,Cum_TPV,...,%D,ATR_signal,Buy_D,Sell_D,Buy_MACD,Sell_MACD,Buy_VWAP,Sell_VWAP,Buy_bollinger,Sell_bollinger
Date,,,,,,,,,,,,,,,,,,,,,
2022-02-28 00:00:00-05:00,1565.699951,1690.650024,1553.204956,1670.250000,1109,0.0,0.0,1638.034993,1.816581e+06,1.104658e+08,...,NaN,0,0,0,0,0,0,0,0,0
2022-03-01 00:00:00-05:00,1680.449951,1708.500000,1542.750000,1700.849976,742,0.0,0.0,1650.699992,1.224819e+06,1.116907e+08,...,NaN,0,0,0,0,0,0,0,0,0
2022-03-02 00:00:00-05:00,1728.900024,1848.750000,1696.770020,1787.550049,1848,0.0,0.0,1777.690023,3.285171e+06,1.149758e+08,...,NaN,0,0,0,0,0,0,0,0,0
2022-03-03 00:00:00-05:00,1810.500000,1818.150024,1665.150024,1683.000000,673,0.0,0.0,1722.100016,1.158973e+06,1.161348e+08,...,NaN,0,0,0,0,0,0,0,0,0
2022-03-04 00:00:00-05:00,1652.400024,1698.300049,1545.300049,1583.550049,629,0.0,0.0,1609.050049,1.012092e+06,1.171469e+08,...,NaN,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-12-23 00:00:00-05:00,3.840000,5.110000,2.810000,3.110000,105557300,0.0,0.0,3.676667,3.880990e+08,1.172929e+09,...,19.684470,1,1,0,0,0,0,0,1,0
2024-12-24 00:00:00-05:00,3.000000,3.130000,2.450000,2.530000,1590800,0.0,0.0,2.703333,4.300463e+06,1.177230e+09,...,29.344477,1,0,0,0,0,0,0,1,0
2024-12-26 00:00:00-05:00,2.430000,2.920000,2.430000,2.660000,739900,0.0,0.0,2.670000,1.975533e+06,1.179205e+09,...,40.448961,1,0,0,0,0,0,0,0,0
